In [128]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report,confusion_matrix , mean_squared_error
from sklearn import tree , datasets
import os
from sklearn import linear_model
from sklearn.metrics import mean_squared_error as mse
import datetime

In [129]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [130]:
# data source path
data_path = r'./data'
final_data_path = r'./final_data'



In [274]:
# data read and rename columns 
birth_col = ['cow_number','confinement date','Date of dried milk','calf number1','calf number2','cow weight','login date'
            ,'Calculated tire','tire','difficulty level of confinement','calf size','calf gender','farm number']

report_col = ['ID','year','month','farm number','cow_number','father_cow_number','mother_cow_number','birthday','tire'
              ,'lactation date','milk support','last confinement date','sampling date','age of month','inspection date'
             ,'final breeding date','final breeding semen','breeding times','before confinement date','first breeding date'
             ,'first breeding semen']

birth_df = pd.read_csv(os.path.join(data_path , 'birth.csv') , names = birth_col,  skiprows=1)
report_df = pd.read_csv(os.path.join(data_path , 'report.csv') , names = report_col , skiprows=1)

# new data get for 11/25 date
report_df = pd.concat([report_df, pd.read_csv(os.path.join(final_data_path , 'report.csv') , names = report_col , skiprows=1)])

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [53]:
print('birth table : ')
print ("Dataset Shape:: ", birth_df.shape)
birth_df.head()

birth table : 
Dataset Shape::  (3761, 13)


,cow_number,confinement date,Date of dried milk,calf number1,calf number2,cow weight,login date,Calculated tire,tire,difficulty level of confinement,calf size,calf gender,farm number
0,87121677,2013/1/4 00:00,NaN,NaN,NaN,NaN,2013/1/25 00:00,True,2,1.0,NaN,NaN,B
1,96126940,2013/1/2 00:00,NaN,NaN,NaN,522.0,2013/1/25 00:00,True,2,2.0,NaN,NaN,B
2,95181708,2013/1/14 00:00,2014/1/3 00:00,2188113,NaN,670.0,2013/1/25 00:00,True,5,1.0,NaN,NaN,C
3,97127665,2013/1/9 00:00,2014/3/12 00:00,NaN,NaN,NaN,2013/1/25 00:00,True,3,2.0,NaN,NaN,B
4,97182634,2013/1/9 00:00,2014/3/25 00:00,NaN,NaN,630.0,2013/1/25 00:00,True,4,1.0,NaN,NaN,C


In [85]:
print('report table : ')
print ("Dataset Shape:: ", report_df.shape)
report_df.head()

report table : 
Dataset Shape::  (39946, 21)


,ID,year,month,farm number,cow_number,father_cow_number,mother_cow_number,birthday,tire,lactation date,...,last confinement date,sampling date,age of month,inspection date,final breeding date,final breeding semen,breeding times,before confinement date,first breeding date,first breeding semen
0,1,2013,3,A,52612,507H7173,96040545,2011/1/1 00:00,1,78.0,...,2013/1/1 00:00,2013/3/20 00:00,26,2013/3/21 00:00,2015/5/5 00:00,507H11621,1,NaN,2013/3/5 00:00,501H8784
1,2,2013,4,A,52612,507H7173,96040545,2011/1/1 00:00,1,107.0,...,2013/1/1 00:00,2013/4/18 00:00,27,2013/4/19 00:00,2015/5/5 00:00,507H11621,1,NaN,2013/3/5 00:00,501H8784
2,3,2013,5,A,52612,507H7173,96040545,2011/1/1 00:00,1,140.0,...,2013/1/1 00:00,2013/5/21 00:00,28,2013/5/22 00:00,2015/5/5 00:00,507H11621,2,NaN,2013/3/5 00:00,501H8784
3,4,2013,6,A,52612,507H7173,96040545,2011/1/1 00:00,1,175.0,...,2013/1/1 00:00,2013/6/25 00:00,29,2013/6/26 00:00,2015/5/5 00:00,507H11621,2,NaN,2013/3/5 00:00,501H8784
4,5,2013,7,A,52612,507H7173,96040545,2011/1/1 00:00,1,210.0,...,2013/1/1 00:00,2013/7/30 00:00,30,2013/7/31 00:00,2015/5/5 00:00,507H11621,2,NaN,2013/3/5 00:00,501H8784


In [275]:
# change string to int 
def process_time_format_sampling_date(x):
    #early start with 2000/6/26 00:00
    # col = birthday to days
    date = datetime.datetime.strptime(x['sampling date'] , "%Y/%m/%d %H:%M")
    
    birthday_days = date-datetime.datetime.strptime('2013/1/1 00:00' , "%Y/%m/%d %H:%M")
    return int(birthday_days.days/30)
def replace_string_int(x , items):
    for i,j in items:
        if x == j :
            return str(i) 

def change_col(col_list,report_df):
    for col in col_list:
        farm_number_list = list(set(report_df[col]))
        turple_fram_number = [i for i in enumerate(farm_number_list)]
        #print(turple_fram_number)
        report_df[col] = report_df[col].apply(lambda x : replace_string_int(x , turple_fram_number))
        
    return report_df

def counts_with_months(x):
    return (int(x['year']) - 2013)*12 + int(x['month'])

def RMSE(y_true,y_pred):
    return np.sqrt(np.mean(np.square(y_true-y_pred)))


change_col_list = ['farm number','final breeding semen','first breeding semen','father_cow_number','mother_cow_number']
#report_df =report_df.dropna(subset=['farm number','final breeding semen','first breeding semen','father_cow_number','mother_cow_number'])
report_df = change_col(change_col_list , report_df)




In [276]:

# generator time series 
report_df['months'] = report_df.apply(lambda x : counts_with_months(x) , 1)
#report_df['sampling_date_to_days'] = report_df.apply(lambda x : process_time_format_sampling_date(x),1)


In [283]:
save_model_group={}
temp_df = report_df.drop(['year','month','birthday','lactation date','last confinement date','sampling date','inspection date'
                   ,'final breeding date','before confinement date','first breeding date'],axis=1)
temp_df_check_ = temp_df
temp_df = temp_df.dropna()
for group in set(report_df['farm number']):
    
    print('this group farm = {}'.format(group))
    x = temp_df[temp_df['farm number'] == group].drop(['milk support'],axis=1)
    y = temp_df[temp_df['farm number'] == group]['milk support']

    
    for index in set(x['breeding times']):
        
        x_breed = x[x['breeding times'] == index]
        y_breed = y[x_breed.index]
        x_train, x_test, y_train, y_test = train_test_split( x_breed, y_breed, test_size = 0.1, random_state = 10)
        #print('x train shape {}'.format(x_train.shape))
        #print('x test shape {}'.format(x_test.shape))
        #print('y train shape {}'.format(y_train.shape))
        #print('y test shape {}'.format(y_test.shape))

        '''
        Index(['ID', 'farm number', 'cow_number', 'father_cow_number',
               'mother_cow_number', 'tire', 'milk support', 'age of month',
               'final breeding semen', 'breeding times^^', 'first breeding semen',
               'months'],
              dtype='object')

        '''
        # Decision Tree Regression + Bagging Regressor
        from sklearn import tree
        from sklearn.ensemble import BaggingRegressor
        #tree.DecisionTreeRegressor()
        model = BaggingRegressor(tree.DecisionTreeRegressor(criterion='mae',splitter='random' ,max_features='sqrt'),n_estimators=300, random_state=42,n_jobs=3
                                )

        #model.fit(x_train,y_train)
        model.fit(x_breed,y_breed)
        #y_pred = model.predict(x_test)
        save_model_group.update({str(group) + "-" + str(index):model})
        if group == '2':
            print('Bagging Regressor + Decision Tree Regression  with RMSE is %.2f -4.63 score' %RMSE(y_pred ,y_test))
        elif group == '0':
            print('Bagging Regressor + Decision Tree Regression  with RMSE is %.2f -6.09score' %RMSE(y_pred ,y_test))
        elif group == '1' :
            print('Bagging Regressor + Decision Tree Regression  with RMSE is %.2f -2.99score' %RMSE(y_pred ,y_test))
        else :
            print('Bagging Regressor + Decision Tree Regression  with RMSE is %.2f score' %RMSE(y_pred ,y_test))

this group farm = 2
Bagging Regressor + Decision Tree Regression  with RMSE is 20.91 -4.63 score
Bagging Regressor + Decision Tree Regression  with RMSE is 17.76 -4.63 score
Bagging Regressor + Decision Tree Regression  with RMSE is 15.94 -4.63 score
Bagging Regressor + Decision Tree Regression  with RMSE is 14.17 -4.63 score
Bagging Regressor + Decision Tree Regression  with RMSE is 13.50 -4.63 score
Bagging Regressor + Decision Tree Regression  with RMSE is 12.62 -4.63 score
Bagging Regressor + Decision Tree Regression  with RMSE is 10.17 -4.63 score
Bagging Regressor + Decision Tree Regression  with RMSE is 11.09 -4.63 score
Bagging Regressor + Decision Tree Regression  with RMSE is 9.20 -4.63 score
Bagging Regressor + Decision Tree Regression  with RMSE is 8.95 -4.63 score
Bagging Regressor + Decision Tree Regression  with RMSE is 6.75 -4.63 score
Bagging Regressor + Decision Tree Regression  with RMSE is 3.16 -4.63 score
Bagging Regressor + Decision Tree Regression  with RMSE is 8

In [58]:
temp_df_check_.columns

Index(['ID', 'farm number', 'cow_number', 'father_cow_number',
       'mother_cow_number', 'tire', 'milk support', 'age of month',
       'final breeding semen', 'breeding times', 'first breeding semen',
       'months'],
      dtype='object')

In [64]:
# Decision Tree Regression + Bagging Regressor
from sklearn import tree
from sklearn.ensemble import BaggingRegressor
#tree.DecisionTreeRegressor()
model = BaggingRegressor(tree.DecisionTreeRegressor(),n_estimators=10, random_state=42,n_jobs=3
                        )
model.fit(x_train,y_train)
y_pred = model.predict(x_test)

print('Bagging Regressor + Decision Tree Regression  with RMSE is %.2f score' %RMSE(y_pred ,y_test))


Bagging Regressor + Decision Tree Regression  with RMSE is 5.31 score


In [ ]:
submit_data = report_df[report_df['milk support'].isna()]
submit_data.fillna(method='ffill', inplace=True)
final_test_x = submit_data.drop(['year','month','birthday','lactation date','last confinement date','sampling date','inspection date'
                   ,'final breeding date','before confinement date','first breeding date','milk support'
                                ],axis=1)
submit_data['milk support'] = model.predict(final_test_x)
submit_data['milk support'] = submit_data['milk support'].apply(lambda x : round(x,2))

In [ ]:
submit_file = pd.read_csv(os.path.join(final_data_path,'submission.csv'))
submit_file['1'] = list(submit_data['milk support'])
submit_file.to_csv(os.path.join(final_data_path , 'submission_1.csv'),index=False)

In [195]:
# one feature
submit_data = report_df[report_df['milk support'].isna()]
submit_data.fillna(method='ffill', inplace=True)
final_test_x = submit_data.drop(['year','month','birthday','lactation date','last confinement date','sampling date','inspection date'
                   ,'final breeding date','before confinement date','first breeding date','milk support'
                                ],axis=1)
concat_by_submitdata = pd.DataFrame()
for index , model in save_model_group.items():
    test_forloop = final_test_x[(final_test_x['breeding times'] == (index))]
    split_by_froup = submit_data[(submit_data['breeding times'] == (index))]

    if  split_by_froup.empty == False and test_forloop.empty== False:
        split_by_froup['milk support'] = model.predict(test_forloop)
        concat_by_submitdata = pd.concat([concat_by_submitdata, split_by_froup])
concat_by_submitdata['milk support'] = concat_by_submitdata['milk support'].apply(lambda x : round(x,2))

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:3790: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  downcast=downcast, **kwargs)
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view

In [284]:
# double two feature
col = 'breeding times'
submit_data = report_df[report_df['milk support'].isna()]
submit_data.fillna(method='ffill', inplace=True)
final_test_x = submit_data.drop(['year','month','birthday','lactation date','last confinement date','sampling date','inspection date'
                   ,'final breeding date','before confinement date','first breeding date','milk support'
                                ],axis=1)
concat_by_submitdata = pd.DataFrame()
for index , model in save_model_group.items():
    group,breed = index.split('-')
    print(group,breed)
    test_forloop = final_test_x[(final_test_x['farm number'] == (group)) & (final_test_x[col] == int(breed))]
    split_by_froup = submit_data[(submit_data['farm number'] == (group)) & (submit_data[col] == int(breed))]
    #print(len(group),len(breed))
    if  split_by_froup.empty == False and test_forloop.empty== False:
        split_by_froup['milk support'] = model.predict(test_forloop)
        concat_by_submitdata = pd.concat([concat_by_submitdata, split_by_froup])
concat_by_submitdata['milk support'] = concat_by_submitdata['milk support'].apply(lambda x : round(x,2))

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:3790: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  downcast=downcast, **kwargs)


0 8


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


2 1


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


...

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


0 0


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


0 11


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [285]:
concat_by_submitdata =concat_by_submitdata.sort_values(by='ID')
submit_file = pd.read_csv(os.path.join(final_data_path,'submission.csv'))
submit_file['1'] = list(concat_by_submitdata['milk support'])

submit_file.to_csv(os.path.join(final_data_path , 'submission_1.csv'),index=False)
print('save as {}'.format(os.path.join(final_data_path , 'submission_1.csv')))

save as ./final_data\submission_1.csv


In [263]:
len(submit_file)

6692

In [268]:
len(concat_by_submitdata)

6692